# 기본 설정

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import json
import folium

In [3]:
# 한글 문제
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

## 데이터 불러오기

In [5]:
brfss = pd.read_csv("../data/brfss.csv")
checkup = pd.read_csv("../data/checkup.csv")
medical = pd.read_csv("../data/medical.csv")
region_heart = pd.read_csv("../data/region_heart.csv")

## geojson 불러오기

geojson 출처: https://cartographyvectors.com/map/1516-south-korea-with-regions

In [7]:
with open("../data/south-korea-with-regions_1516.geojson", "r", encoding="UTF-8") as f:
    geo_json = json.load(f)

### geojson에 맞춘 지역 이름

In [8]:
region = {"강원": "강원도", "경기": "경기도", "경남": "경상남도", "경북": "경상북도", 
          "전남": "전라남도", "전북": "전라북도", "충남": "충청남도", "충북": "충청북도", 
          "광주": "광주광역시", "대구": "대구광역시", "대전": "대전광역시", "부산": "부산광역시", 
          "서울": "서울특별시", "울산": "울산광역시", "인천": "인천광역시", 
          "세종": "세종특별자치시", "제주": "제주특별자치도"}

In [10]:
checkup["지역"].replace(region, inplace=True)
medical["지역"].replace(region, inplace=True)

# 함수

In [13]:
def chroplethmap(data, col, color, path=None):
    korea_map = folium.Map(location=[36.36109, 127.79209], zoom_start=6)

    fmap = folium.Choropleth(geo_data=geo_json,
                             data=data,
                             columns=["지역", col],
                             key_on="feature.properties.name",
                             fill_color=color,
                             legend_name=col).add_to(korea_map)

    fmap.geojson.zoom_on_click = False
    fmap.geojson.add_child(folium.features.GeoJsonTooltip(["name"],labels=False))
    display(korea_map)
    
    if path:
        korea_map.save(path)

# 지역별 심장질환

In [11]:
region_heart.columns

Index(['지역', '사망자', '심장장애'], dtype='object')

In [14]:
chroplethmap(region_heart, "사망자", "PuBu")

In [15]:
chroplethmap(region_heart, "심장장애", "PuBu")

# 지역별 의료 수준

In [17]:
medical.columns

Index(['지역', '상급종합병원', '종합병원', '병원', '심장혈관흉부외과', '심장혈관흉부외과.1', '일반의', '전문의',
       '간호사', '임상병리사', '심장혈관흉부외과.2', '심장혈관흉부외과.3', '일반병실', '일반병상', '중환병실',
       '중환병상', '구급차 (대)', '구급대원수'],
      dtype='object')

In [25]:
for col in medical.columns[1:]:
    print("="*20)
    print(col)
    print("="*20)
    chroplethmap(medical, col, "YlGn")
    

상급종합병원


종합병원


병원


심장혈관흉부외과


심장혈관흉부외과.1


...

일반병상


중환병실


중환병상


구급차 (대)


구급대원수
